In [237]:
from typing import List

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_table

import plotly.graph_objs as go
import plotly.plotly as py
import plotly.figure_factory as ff

from flask_cors import CORS

import numpy as np
import pandas as pd
from pandas import Series
from pandas import DataFrame

import os
import requests
import io

import datetime

In [238]:
downweight_granularity=['dtime','Direction','dday','month']

In [239]:
url='https://raw.githubusercontent.com/dkremlg/Booking-Curves/master/GUI_in.csv'
s=requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('utf-8')),sep=',')

df['Dprio']=df['Dprio'].astype('float')
df['Actual Bookings']=df['Actual Bookings'].astype('float')
df['Ramp-up frontier']=df['Ramp-up frontier'].astype('float')
df['Ideal curve (80% LF)']=df['Ideal curve (80% LF)'].astype('float')
df['Ideal curve (100% LF)']=df['Ideal curve (100% LF)'].astype('float')
df['Phase-down frontier']=df['Phase-down frontier'].astype('float')
df['Group Pax']=df['Group Pax'].astype('float')

url='https://raw.githubusercontent.com/dkremlg/Booking-Curves/master/GUI_in2.csv'
s=requests.get(url).content
df2=pd.read_csv(io.StringIO(s.decode('utf-8')),sep=',')

df2=df2.rename(columns={'Spoilage Risk': 'Risk LF below 80%'})

df2['Dprio']=df2['Dprio'].astype('float')
df2['NumPax']=df2['NumPax'].astype('float')
df2['SpoilageRisk']=df2['SpoilageRisk'].astype('float')
df2['SpillageRisk']=df2['SpillageRisk'].astype('float')
df2['Intensity_full']=df2['Intensity_full'].astype('float')
df2['Intensity_downweighted']=df2['Intensity_downweighted'].astype('float')

df2=df2.sort_values(by=['DepDate','Dprio'])

df2['SpoilageRisk']=df2['SpoilageRisk'].apply(lambda x: float(str(str(x).split('.')[0])+'.'+str(str(x).split('.')[1])[0:2]))
df2['SpillageRisk']=df2['SpillageRisk'].apply(lambda x: float(str(str(x).split('.')[0])+'.'+str(str(x).split('.')[1])[0:2]))
df2['Intensity_full']=df2['Intensity_full'].apply(lambda x: float(str(str(x).split('.')[0])+'.'+str(str(x).split('.')[1])[0:2]))
df2['Intensity_downweighted']=df2['Intensity_downweighted'].apply(lambda x: float(str(str(x).split('.')[0])+'.'+str(str(x).split('.')[1])[0:2]))

df2['DepDate']=df2['DepDate'].apply(lambda x: pd.to_datetime(x))
df2['IssueDate']=df2['DepDate']-df2['Dprio'].apply(lambda x: datetime.timedelta(x-1))
df2=df2.loc[df2['IssueDate']==pd.to_datetime('2019-04-11'),:]
df2['DepDate']=df2['DepDate'].astype('str')
df2['DepDate']=df2[[x for x in df2.columns if x!='IssueDate']]

In [240]:
df2=df2[['DepDate','dtime','Direction','month','dday','SpoilageRisk','Intensity_downweighted','Intensity_full','SpillageRisk']]

In [241]:
url='https://raw.githubusercontent.com/dkremlg/Booking-Curves/master/GUI_in3.csv'
s=requests.get(url).content
df3=pd.read_csv(io.StringIO(s.decode('utf-8')),sep=',')

In [242]:
df3['Dprio']=df3['Dprio'].astype('float')                               
df3['ActualRevenue_full']=df3['ActualRevenue_full'].astype('float')
df3['ActualRevenue_downweighted']=df3['ActualRevenue_downweighted'].astype('float')
df3['TotalRevenue_full']=df3['TotalRevenue_full'].astype('float')
df3['TotalRevenue_downweighted']=df3['TotalRevenue_downweighted'].astype('float')

In [243]:
MonthMapping=DataFrame([[x for x in range(1,13)],['January','February','March','April','May','June','July','August',
                'September','October','November','December']]).transpose()
MonthMapping.columns=['month','Month']
MonthMapping['month']=MonthMapping['month'].astype('int') 
df2=df2.merge(MonthMapping,on='month')
df2['month']=df2['Month']
df2=df2[[x for x in df2.columns if x!='Month']]

In [244]:
IdealCurve=pd.read_csv('IdealCurve_forPlots.csv')

deptime=[str(x/60).split('.')[0]+':'+str(round(float('0.'+str(x/60).split('.')[1])*60)) for x in IdealCurve['dtime'].unique()]
deptime=[x+'0' if len(x.split(':')[1])==1 else x for x in deptime]
deptime=['0'+x if len(x.split(':')[0])==1 else x for x in deptime]

Map_DepTime=DataFrame([IdealCurve['dtime'].unique(),[x+'0' if len(x.split(':')[1])==1 else x for x in deptime]]).transpose()
Map_DepTime.columns=['dtime','deptime']
Map_DepTime['dtime']=Map_DepTime['dtime'].astype('int')

IdealCurve=IdealCurve.merge(Map_DepTime,on='dtime')
IdealCurve['dtime']=IdealCurve['deptime']
IdealCurve=IdealCurve[[x for x in IdealCurve.columns if x!='deptime']]

IdealCurve=IdealCurve.rename(columns=dict(zip(['Ideal_lower','Ideal_average_downweighted','Ideal_average_full','Ideal_upper'],
                                  ['Ramp-up frontier','Ideal curve (80% LF)','Ideal curve (100% LF)','Phase-down frontier'])))

In [245]:
Capacity=pd.read_csv('Capacity_forPlots.csv')

# Import external stylesheet

In [246]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# Vectors for dropdown menues

In [247]:
dep_dates = sorted(df['DepDate'].unique())
dtime = sorted(df['dtime'].unique())
direction=['O','I']
dday=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday']
month=df2['month'].unique().tolist()

# Layout

In [248]:
line_columns_Pax = ['Actual Bookings']
linecolor_dict_Pax = dict(zip(line_columns_Pax,['black']))
linedash_dict_Pax = dict(zip(line_columns_Pax,['solid']))
name_dict_Pax = dict(zip(line_columns_Pax,['Actual Bookings']))

In [249]:
line_columns_Ideal = ['Ramp-up frontier', 'Ideal curve (80% LF)', 'Ideal curve (100% LF)', 
                    'Phase-down frontier']
linecolor_dict_Ideal = dict(zip(line_columns_Ideal,['red', 'blue', 'green', '#E0115F']))
linedash_dict_Ideal = dict(zip(line_columns_Ideal,['dot', 'solid', 'solid', 'dot']))
name_dict_Ideal = dict(zip(line_columns_Ideal,['Ramp-up frontier', 'Ideal curve (80% LF)', 
                        'Ideal curve (100% LF)', 'Phase-down frontier']))

In [250]:
line_columns_Capacity = ['Cabin Capacity']
linecolor_dict_Capacity = dict(zip(line_columns_Capacity,['lightgrey']))
linedash_dict_Capacity = dict(zip(line_columns_Capacity,['solid']))
name_dict_Capacity = dict(zip(line_columns_Capacity,['Cabin Capacity']))

In [251]:
line_columns_Revenue = ['ActualRevenue_full','TotalRevenue_full']
linecolor_dict_Revenue = dict(zip(line_columns_Revenue,[ 'black','green', 'blue']))
linedash_dict_Revenue = dict(zip(line_columns_Revenue,['solid', 'solid', 'solid']))
name_dict_Revenue = dict(zip(line_columns_Revenue,['ActualRevenue_full','TotalRevenue_full']))

In [252]:
rename_dict=dict(zip(df2.columns,
['Departure Date', 'Departure Time', 'Direction', 'Month', 'Weekday', 'Spoilage Risk', 'Intensity (80% LF)', 'Intensity (100% LF)', 'Spillage Risk']))

In [253]:
PAGE_SIZE = 100

In [254]:
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    
    # MAIN HEADER
    html.Div([html.H1(children='Booking Curve Steering System',
    style={
            'textAlign': 'center',
            'color': 'black',
            'margin-bottom': '0.5em',
            'padding' : '50px' ,
            'backgroundColor' : '#66b3cc',
            'width': '90%',
            'display': 'inline-block'
        }),html.Img(src=app.get_asset_url('LuxairGroup_logo.jpg'), 
              style = {'backgroundColor' : '#66b3cc', 
                       'width': '5%', 
                       'height': '6em', 
                       'display': 'inline-block',
                       'margin-left': '0em',
                       'margin-top': '1em'
                      })
        ]),
    
    # CHECKLISTS AND DROPDOWNS
    
        # CHECKLISTS
    
    html.Div([

    html.Div([html.Label('Departure Time', style={'textAlign': 'center'}), 
    dcc.Checklist(id = 'checklist_table_dtime',
    options=[{
    'label': i,
    'value': i
    } for i in dtime],
    values=['06:45'])       
             ],   
    style={'width': '10%', 'display': 'inline-block', 'margin-bottom': '1.25em'}),    

    html.Div([html.Label('Weekday', style={'textAlign': 'center'}), 
    dcc.Checklist(id = 'checklist_table_dday',
    options=[{
    'label': i,
    'value': i
    } for i in dday],
    values=['Tuesday'])        
             ],
    style={'width': '10%', 'display': 'inline-block', 'margin-bottom': '1.25em'}),     
        
    html.Div([html.Label('Month', style={'textAlign': 'center'}), 
    dcc.Checklist(id = 'checklist_table_month',
    options=[{
    'label': i,
    'value': i
    } for i in month],
    values=['April'])        
             ],
    style={'width': '10%', 'display': 'inline-block', 'margin-bottom': '1.25em'})
        
    ]),
        
    # DROPDOWNS FOR DEPARTURE    
    
        # DEPARTURE DATE    
    
    html.Div([
    
    html.Div([html.Label('Departure Date', style={'textAlign': 'center'}), 
    dcc.Dropdown(id = 'dropdown_depdates',
    options=[{
    'label': i,
    'value': i
    } for i in dep_dates],
    placeholder="Select a departure date",                                 
    value='All Departure Dates')],
    style={'width': '10%', 'display': 'inline-block', 'margin-left': '90em', 'margin-bottom': '0.25em'}),
       
        # DEPARTURE TIME    
        
    html.Div([html.Label('Departure Time', style={'textAlign': 'center'}),
    dcc.Dropdown(id = 'dropdown_dtime',
    options=[{
    'label': i,
    'value': i
    } for i in dtime],
    placeholder="Select a departure time",                       
    value='All Departure Times')],
    style={'width': '10%', 'display': 'inline-block',  'margin-left': '2.5em', 'margin-bottom': '0.25em'})        
             
    ],
    
    style={'margin-top': '-10em'}),
    
    # CONTAINERS
    
        # TABLE CONTAINER
    html.Div([html.Div([
        
    dash_table.DataTable(
            id = 'booking_table',
            columns=[{'id': c, 'name': rename_dict[c], 'deletable': True} for c in df2.columns],
            style_header={'backgroundColor': 'rgb(30, 30, 30)','color': 'white', 'textAlign': 'center'},

             style_data_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(40, 40, 40)',
                'color': 'white',
                'textAlign': 'center'
            },
            {
               'if': {'row_index': 'odd', 'column_id': 'Dprio'},
                 'backgroundColor': '#568dba',
                'color': 'black',
                'textAlign': 'center'
             },    
            {
                'if': {'row_index': 'even'},
                'backgroundColor': 'rgb(50, 50, 50)',
                'color': 'white',
                'textAlign': 'center'
            },
            {
                'if': {'row_index': 'even', 'column_id': 'Dprio'},
                'backgroundColor': '#6798c1',
                'color': 'black',
                 'textAlign': 'center'
            },     
            {
                 'if': {
                     'column_id': 'SpoilageRisk',
                     'filter': 'SpoilageRisk >= num(0.95)',
                 },
                 'color': 'red',
            },
            {
                'if': {
                     'column_id': 'SpillageRisk',
                     'filter': 'SpillageRisk >= num(0.95)',
                 },
                'color': '#E0115F'
            }
            ],
            style_cell={
            # all three widths are needed
            'minWidth': '50px', 'width': '50px', 'maxWidth': '50px',
            'whiteSpace': 'normal'
        },

    pagination_settings={
        'current_page': 0,
        'page_size': PAGE_SIZE
    },        
        
    pagination_mode='be',

    filtering='be',
    filtering_settings='',

    sorting='be',
    sorting_type='multi',
    sorting_settings=[]

            )
                       ],
        
        style={'width': '52.5%', 'display': 'inline-block', 'margin-top': '5em'}),
    
        # GRAPH CONTAINER                   
    html.Div([dcc.Tabs(id="tabs", children=[
        dcc.Tab(label='Pax', children=[
            html.Div([
                dcc.Graph(
                    id='graph-container_pax')
                ])
            ]),
        dcc.Tab(label='Revenue', children=[
            html.Div([
                dcc.Graph(
                    id='graph-container_rev')
                ])
            ])])
    ],style={'width': '47.5%', 'float': 'right', 'display': 'inline-block'})
             
             ]),
    
])

In [ ]:
# @app.callback(
#     Output('booking_table', 'data'),
#     [Input('checklist_table_dtime', 'values'),
#      Input('checklist_table_dday', 'values'), 
#      Input('checklist_table_month', 'values')])
# def filter_df2(values_dtime,values_dday,values_month):
#     global dff2
#     dff2 = df2.loc[df2['dtime'].apply(lambda x: x in values_dtime)\
#                        &df2['dday'].apply(lambda x: x in values_dday)\
#                        &df2['month'].apply(lambda x: x in values_month),:]

#     return dff2.to_dict('rows')  

In [ ]:
@app.callback(
    Output('booking_table', 'data'),
    [Input('checklist_table_dtime', 'values'),
     Input('checklist_table_dday', 'values'), 
     Input('checklist_table_month', 'values'),
     Input('booking_table', 'pagination_settings'),
     Input('booking_table', 'sorting_settings'),
     Input('booking_table', 'filtering_settings'),
    ])
def update_graph(values_dtime, values_dday, values_month, pagination_settings, sorting_settings, filtering_settings):
    
    filtering_expressions = filtering_settings.split(' && ')

    dff2 = df2.loc[df2['dtime'].apply(lambda x: x in values_dtime)\
                       &df2['dday'].apply(lambda x: x in values_dday)\
                       &df2['month'].apply(lambda x: x in values_month),:]
    
    for filter in filtering_expressions:
        if ' eq ' in filter:
            col_name = filter.split(' eq ')[0].replace("\"","")
            filter_value = filter.split(' eq ')[1]
            dff2 = dff2.loc[dff2[col_name] == filter_value]
        if ' > ' in filter:
            col_name = filter.split(' > ')[0].replace("\"","")
            filter_value = float(filter.split(' > ')[1])
            dff2 = dff2.loc[dff2[col_name] > filter_value]
        if ' < ' in filter:
            col_name = filter.split(' < ')[0].replace("\"","")
            filter_value = float(filter.split(' < ')[1])
            dff2 = dff2.loc[dff2[col_name] < filter_value]

    if len(sorting_settings):
        dff2 = dff2.sort_values(
            [col['column_id'] for col in sorting_settings],
            ascending=[
                col['direction'] == 'asc'
                for col in sorting_settings
            ],
            inplace=False
        )

    return dff2.iloc[
        pagination_settings['current_page']*pagination_settings['page_size']:
        (pagination_settings['current_page'] + 1)*pagination_settings['page_size']
    ].to_dict('rows')

In [ ]:
@app.callback(
    Output('graph-container_pax', 'figure'),
    [Input('dropdown_depdates', 'value'),
    Input('dropdown_dtime', 'value')])
def update_Paxgraph(value_depdates, value_dtime):

    dff = df.loc[(df['DepDate'] == value_depdates)&(df['dtime'] == value_dtime),:] 
    
    dff2=IdealCurve.loc[(IdealCurve['DepDate'] == value_depdates)&(IdealCurve['dtime'] == value_dtime),:] 
 
    dff3=Capacity.loc[(Capacity['DepDate'] == value_depdates)&(Capacity['dtime'] == value_dtime),:] 

    traces = []
    for i in line_columns_Pax:
        traces.append(go.Scatter(
            x=dff['Dprio'].tolist(),
            y=dff[i].tolist(),
            mode = 'lines',
            name = name_dict_Pax[i],
            line = dict(
              dash = linedash_dict_Pax[i],
              color = linecolor_dict_Pax[i],
              width = 2
           )
    ))
        
    for i in line_columns_Ideal:
        traces.append(go.Scatter(
            x=dff2['Dprio'].tolist(),
            y=dff2[i].tolist(),
            mode = 'lines',
            name = name_dict_Ideal[i],
            line = dict(
              dash = linedash_dict_Ideal[i],
              color = linecolor_dict_Ideal[i],
              width = 2
           )
    ))        

    for i in line_columns_Capacity:
        traces.append(go.Scatter(
            x=dff3['Dprio'].tolist(),
            y=dff3[i].tolist(),
            mode = 'lines',
            name = name_dict_Capacity[i],
            line = dict(
              dash = linedash_dict_Capacity[i],
              color = linecolor_dict_Capacity[i],
              width = 2
           )
    ))          
        
    return {
    'data': traces,
    'layout': go.Layout(
        # xaxis={'title': 'Days prior to departure', 'range': [dff['Dprio'].min(), 365]},
        # yaxis={'title': 'Number of bookings', 'range': [0, max([dff['Phase-down frontier'].max(),dff['Actual Bookings'].max()])]},
        xaxis={'title': 'Days prior to departure', 'range': [0, 365]},
        yaxis={'title': 'Number of bookings', 'range': [0, 70]},
        height=600)
    }

In [ ]:
@app.callback(
 Output('graph-container_rev', 'figure'),
 [Input('dropdown_depdates', 'value'),
 Input('dropdown_dtime', 'value')])
def update_Revgraph(value_depdates, value_dtime):

    dff = df3.loc[(df3['DepDate'] == value_depdates)&(df3['dtime'] == value_dtime),:] # update with your own logic    

    traces = []
    for i in line_columns_Revenue:
        traces.append(go.Scatter(
             x=dff['Dprio'].tolist(),
             y=dff[i].tolist(),
             mode = 'lines',
             name = name_dict_Revenue[i],
             line = dict(
               dash = linedash_dict_Revenue[i],
               color = linecolor_dict_Revenue[i],
               width = 2
            )
     ))

    return {
     'data': traces,
     'layout': go.Layout(
         xaxis={'title': 'Days prior to departure', 'range': [dff['Dprio'].min(), 365]},
         yaxis={'title': 'Number of bookings', 'range': [min([dff['ActualRevenue_downweighted'].min(),dff['TotalRevenue_downweighted'].min()]), 
                                                         max([dff['ActualRevenue_full'].max(),dff['TotalRevenue_full'].max()])]},
         height=600)
 }

In [ ]:
if __name__ == '__main__':
    app.run_server(port=1000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:1000/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Apr/2019 14:04:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2019 14:05:00] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2019 14:05:00] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2019 14:05:00] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2019 14:05:00] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2019 14:05:00] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2019 14:05:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2019 14:05:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2019 14:05:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2019 14:05:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2019 14:05:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2019 14:05:07] "POST /_dash-update-component

127.0.0.1 - - [16/Apr/2019 17:45:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2019 17:45:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2019 17:45:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2019 17:45:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2019 17:45:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2019 17:45:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2019 17:46:00] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2019 17:46:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2019 17:46:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2019 17:46:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2019 17:46:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2019 17:46:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.